In [5]:
# Reddit data_Extraction
# This class containes the code for extracting popularity information related to cities extracted through Reddit API.
# Detailed description availble through inline comments.
import praw
import re
import pandas as pd
from datetime import datetime


# Initialising reddit api through the security tokens
reddit = praw.Reddit(client_id='WwPjoGfdP9dgDw',
                     client_secret='-D-j2d20fDriAhE33lCju193oRs',
                     user_agent='dmdd6210',
                     username='Raju0788',
                     password='Hopeful28!')


# Defining function 'post_related_to_player', which checks if the reddit submission(post) contains cricket player's name and 
# return true or false accordingly
def post_related_to_city(subobj, city_name):
    found_a_match = False
    if (len(city_name)>3 and ((re.search((city_name),subobj.title,re.IGNORECASE)) or (re.search((city_name),subobj.selftext,re.IGNORECASE)))):
        found_a_match = True
    return found_a_match


# Reading the data extracted from kaggle, which contains the list of cities, whose popularity information is what is 
# going to be extracted from twitter  
cities_input = pd.read_csv('City.csv')
#Constructing a dictionary from two columns of the read data namely 'city_id' and 'city_name' adn stored in cricket_players_dict
cities_dict= dict(zip(cities_input.city_id, cities_input.city_name))



# Initialising below empty lists for storing player popularity related information
city_id_list = []
submission_content_list = []
submission_content_title_list = []
submission_datetime_list = []
submission_up_count_list = []
submission_comments_count_list = []
submission_down_count_list = []
user_id_list = []
submission_id_list = []
user_name_list = []
user_post_karma_list = []
user_comment_karma_list = []


#Iterating through the items of dictionary
for key, value in cities_dict.items():
    
    # Using reddit api, retrieving reddit posts from 'Cricket' channel
    submissions = reddit.subreddit('travel').hot(limit=1000)
    for submission in submissions:
        
        # Checkng if the reddit posts contain player name(chekced by the method 'pst_related_to_player')
        if (post_related_to_city(submission,value)):
            
            # Stores the player id in cricket_player_id_list
            city_id_list.append(key)
            # Stores the reddit submisison text in submission_content_list
            submission_content_list.append(submission.selftext)
            # Converting the date into readable format and storing in submission_datetime_list
            submission_datetime_list.append(datetime.fromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'))
            # Storing the related upvotes and downvotes for a reddit submission
            submission_up_count_list.append(submission.ups)
            submission_down_count_list.append(submission.downs)
            
            
            # Stores the submission id in submission_id_list
            submission_id_list.append(submission.id)
            # Stores the submission's author name in user_id_list
            user_id_list.append(submission.author.name)
            # Stores the number of comments written for the submission in submission_comments_count_list 
            submission_comments_count_list.append(submission.num_comments)
            # Stores the title of the submission in submission_content_title_list
            submission_content_title_list.append(submission.title)

            
            
#Creating a dataframe cricketplayer_submission_df to store submission information collected with regard to cricket players               
city_submission_df = pd.DataFrame(data = city_id_list, columns = ['Player_ID'])
city_submission_df['Content'] = submission_content_list  
city_submission_df['Created_Time'] = submission_datetime_list
city_submission_df['Up_vote_Count'] = submission_up_count_list
city_submission_df['Down_vote_Count'] = submission_down_count_list
city_submission_df['Post_ID'] = submission_id_list
city_submission_df['User_ID'] = user_id_list
city_submission_df['Comments_Count'] = submission_comments_count_list
city_submission_df['Content_Title'] = submission_content_title_list



# Removal of duplicate rows with regard to 'Post_ID' in cricketplayer_submission_df
city_submission_df.drop_duplicates(subset=['Post_ID'], keep='first', inplace= True)



# Creating a list, which has all duplicates removed out of user_id_list
test_refined_user_id_list = list(set(user_id_list))



# Initialising below empty lists for storing submisison user related information
submission_count_list = []
comments_count_list = []
submission_score_list = []
comments_score_list = []



#Iterating through the test_refined_user_id_list
for sub_user_id in test_refined_user_id_list:
    #Storing the relevant user relaed to the user id in 'user' variable
    user = reddit.redditor(sub_user_id)

    # Setting all initial paramaters to be zero
    submission_count = 0
    comments_count = 0
    submission_score = 0
    comments_score = 0

    
    
    # Iterating through all submisisons of the user
    for subm in user.submissions.new():
        submission_count = submission_count + 1
        submission_score = submission_score + subm.score

        
        
    # Storing the submission count and submission score in  submission_count_list and submission_score_list   
    submission_count_list.append(submission_count)   
    submission_score_list.append(submission_score)
        
    
    
    # Iterating through all comments of the user
    for comment in user.comments.new():
        comments_count = comments_count + 1
        comments_score = comments_score + comment.score
        
        
    
    # Storing the comments count and comments score in  comments_count_list and comments_score_list       
    comments_count_list.append(comments_count)   
    comments_score_list.append(comments_score)
    
    
    
#Creating a dataframe cricketplayer_submissionuser_df to store user information related to the submisisons which had information
# about cricket players
city_submissionuser_df = pd.DataFrame(data = test_refined_user_id_list, columns = ['User_ID'])
city_submissionuser_df['Total_Number_Of_Posts'] = submission_count_list  
city_submissionuser_df['Total_Post_Score'] = submission_score_list
city_submissionuser_df['Total_Number_Of_Comments_Made'] = comments_count_list
city_submissionuser_df['Total_Comment_Score'] = comments_score_list



# Removing the single and double quotes from 'Content_Title' for easy import into database
city_submission_df['Content_Title'] = city_submission_df['Content_Title'].str.replace('"','')
city_submission_df['Content_Title'] = city_submission_df['Content_Title'].str.replace("'","")



# Removing the single and double quotes from 'Content' for easy import into database
city_submission_df['Content'] = city_submission_df['Content'].str[:400]
city_submission_df['Content'] = city_submission_df['Content_Title'].str.replace('"','')
city_submission_df['Content'] = city_submission_df['Content_Title'].str.replace("'","")



city_submission_df.to_csv("City_Reddit_Extracted_RedditPost.csv", index=False)
city_submissionuser_df.to_csv("City_Reddit_Extracted_RedditUser.csv", index=False)


""""
CONCLUSIONS:

This code uses the list of city names,selected for the project , tries to find the popularity parameters related 
to the same through Reddit API, stores the results in dataframe and exports it to a CSV file.

CONTRIBUTIONS:

RAJENDRA KUMAR RAJKUMAR - 50% 
MONISH  HIRISAVE RAGHU - 50%

CITATIONS:

1. https://www.geeksforgeeks.org
2. https://github.com/nikbearbrown/INFO_6210
3. Reddit developer docs

The code with regard to extraction of information from reddit was used from the above mentioned resources

Original writtten code - 90%
Code referenced from external sources(but modified suiting needs) - 10%

LICENSE:

Copyright <2019> <RAJENDRA KUMAR RAJKUMAR, MONISH  HIRISAVE RAGHU>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


"""

'"\nCONCLUSIONS:\n\nThis code uses the list of city names,selected for the project , tries to find the popularity parameters related \nto the same through Reddit API, stores the results in dataframe and exports it to a CSV file.\n\nCONTRIBUTIONS:\n\nRAJENDRA KUMAR RAJKUMAR - 50% \nMONISH  HIRISAVE RAGHU - 50%\n\nCITATIONS:\n\n1. https://www.geeksforgeeks.org\n2. https://github.com/nikbearbrown/INFO_6210\n3. Reddit developer docs\n\nThe code with regard to extraction of information from reddit was used from the above mentioned resources\n\nOriginal writtten code - 90%\nCode referenced from external sources(but modified suiting needs) - 10%\n\nLICENSE:\n\nCopyright <2019> <RAJENDRA KUMAR RAJKUMAR, MONISH  HIRISAVE RAGHU>\n\nPermission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish